In [ ]:
import cv2
import face_recognition
import pickle
import matplotlib.pyplot as plt
import os

# Dirección RTSP de la cámara (modificá esta línea con tus datos reales de usuario, IP y puerto)
camara_rtsp = "rtsp://usuario:contraseña@IP:PUERTO/stream1"

# 📂 Cargar embeddings de rostros conocidos
embeddings_dir = "embeddings/"
known_face_encodings = []
known_face_names = []

for filename in os.listdir(embeddings_dir):
    if filename.endswith(".pkl"):
        filepath = os.path.join(embeddings_dir, filename)
        with open(filepath, "rb") as f:
            data = pickle.load(f)
            known_face_encodings.extend(data["encodings"])
            known_face_names.extend([data["name"]] * len(data["encodings"]))

# 📷 Capturar un frame desde la cámara Tapo
cap = cv2.VideoCapture(camara_rtsp)
ret, frame = cap.read()
cap.release()

if not ret:
    raise RuntimeError("❌ Error al capturar imagen desde la Tapo.")

# 🔎 Procesar la imagen con face_recognition
rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
face_locations = face_recognition.face_locations(rgb_frame)
face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

nombres_detectados = []

for face_encoding, face_location in zip(face_encodings, face_locations):
    matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.4)
    name = "Desconocido"
    if True in matches:
        matched_index = matches.index(True)
        name = known_face_names[matched_index]

    if name not in nombres_detectados or name != "Desconocido":
        nombres_detectados.append(name)

    top, right, bottom, left = face_location
    cv2.rectangle(rgb_frame, (left, top), (right, bottom), (0, 255, 0), 2)
    cv2.putText(rgb_frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

# 📊 Mostrar la imagen en la celda Jupyter
plt.imshow(rgb_frame)
plt.axis("off")
plt.title(f"Reconocimiento Facial: {', '.join(nombres_detectados) or 'Sin detecciones'}")
plt.show()